# Opis działania
Funkcja pobiera tekst który ma przetworzyć w celu znalezienie danych wrażliwych i czy tekst zawiera nieodpowiednie treści
1. Tekst jest wysyłany do API OpenAI wraz z odpowiednim zapytaniem. Zwracana jest lista zawierające kategorie i odpowiadające im dane wrażliwe. Jest też zwracana informacja o nieodpowiednich treściach
2. Usuwanie edge case
3. Model nie zawsze zwraca poprawne odpowiedzi czy występują nieodpowiednie treści więc odpowiedź i/lub tekst jest ponownie przetwarzany w celu sprawdzenia poprawności
4. Wyświetlana jest wiadomość czy treść zawiera dane wrażliwe i nieodpowiednie treści
Przy każdorazowym sprawdzeniu tekstu aplikacja wyśle 1-3 zapytania

#### def ochrona(user_message)
To jest główna funkcja programu, która przetwarza wiadomość użytkownika i zwraca trzy wartości:
- listę tupli zawierającą dane wrażliwe i ich kategorie
- oryginalną wiadomość
- oryginalną wiadomość z zastąpionymi danymi

#### def double_check(user_message=user_message, data_to_check=None)
Ta funkcja sprawdza, czy to, co wypisał model rzeczywiście się znajduje w tekście i zwraca "Tak" lub "Nie.
Jeśli odpowiedź nie zostanie podana (data_to_check) to model przeszuka tekst w poszukiwaniu nieodpowiednich treści, a następnie one zostaną sprawdzone.
Zwraca "True" lub "False"

In [9]:
!pip install openai

In [10]:
from logins import OpenAI_key  # ta funkcja zwraca klucz do API
from openai import OpenAI
import os

In [11]:
def ochrona(user_message):
    def check_edge_case(item):
        if len(item) < 2 or "n/a" in item:  # edge case
            return True


    def double_check(user_message=user_message, data_to_check=None):
        # osobne sprawdzenie, czy rzeczywiście nie ma nieodpowiednich treści w tekście
        if data_to_check is None:
            system_message = f"Twoim zdaniem jest sprawdzenie czy w tekście znajdują się nieodpowiednie treści takei jak: mowę nienawiści, groźby, zastraszanie, agresywne zachowanie, niedozwolone czynności itp. Jeśli coś znajdziesz to wypisz je w formacie: 'Nieodpowiednie treści: (wypisz po przecinku co znalazłeś)"

            messages = [{"role": "system", "content": system_message},
                        {"role": "user", "content": user_message}, ]

            completion = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo", temperature=0)
            answer = completion.choices[0].message.content
            data_to_check = (answer.split(":")[0].strip(), answer.split(":")[1].strip())

        # weryfikacja odpowiedzi
        system_message = f"Twoim zdaniem jest sprawdzenie czy w tekście znajdują się takie treści: '{data_to_check[1]}' z kategorii {data_to_check[0]}. Zwróć tylko 'Tak' lub 'Nie'"

        messages = [{"role": "system", "content": system_message},
                    {"role": "user", "content": user_message}, ]

        completion = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo", temperature=0)
        answer = completion.choices[0].message.content
        if "Tak" in answer:
            return True
        else:
            return False

    os.environ["OPENAI_API_KEY"] = OpenAI_key()
    client = OpenAI()

    # prompt, który zwróci listę danych wrażliwych
    system_message = "Twoją odpowiedzią będzie wypisanie listy danych osobowych, których można by było użyć do identyfikacji czyli dane takie jak: imię, nazwisko, adres, nazwy miejsc itp. Jeśli w tekście wykryto mowę nienawiści, groźby, zastraszanie lub inne zakazane czynności to wypisz je po 'Nieodpowiednie treści: '. Każdy z osobnych punktów pisz w nowe linijce. Jeżeli nie będzie żadnych danych do wypisania to napisz 'n/a'.Jeśli jest kilka informacji do tej samej kategorii to wypisz je wszystkie.\nFormat przykładowej odpowiedzi:\nImię: Jan\nNazwisko: Kowalski"
    # przykładowy tekst
    # user_message = "John Smith, urodzony 15 lipca 1985 roku w Nowym Jorku, mieszka na ulicy Maple Avenue 123. Jego numer telefonu to 555-123-4567, a adres e-mail to john.smith@example.com. Wczoraj wieczorem spotkał się z kolegami i zbyt wiele wypił, co skończyło się awanturą. John zachował się bardzo agresywnie i obraźliwie wobec innych osób, co jest niedopuszczalne. W wyniku incydentu został zatrzymany przez policję."

    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content": user_message}, ]

    completion = client.chat.completions.create(messages=messages, model="gpt-3.5-turbo", temperature=0)
    answer = completion.choices[0].message.content

    lista_danych = answer.split("\n")

    for i in range(len((lista_danych))):
        if "Inne: Nieodpowiednie treści:" in lista_danych[i]:
            lista_danych[i] = lista_danych[i].replace("Inne: ", "")
        if ("adres" in lista_danych[i].lower()) and ("ul." in lista_danych[i]):
            lista_danych[i] = lista_danych[i].replace("ul.", "").strip()
        else:
            pass

    # zamiana danych wrażliwych na anonimowe
    zmiana_danych = []
    przetworzony_tekst = user_message

    for item in lista_danych:
        dane_wrazliwe = item.split(":")
        if check_edge_case(item):
            continue
        kategoria = dane_wrazliwe[0].strip()
        dane = dane_wrazliwe[1].strip()
        if "," in dane:  # jeżeli dane zawierają kilka elementów to trzeba je wszystkie zamienić
            temp = dane.split(",")
            for dana in temp:
                przetworzony_tekst = przetworzony_tekst.replace(dana, f"[{kategoria.upper()}]")

        zmiana_danych.append((kategoria, dane))
        przetworzony_tekst = przetworzony_tekst.replace(dane, f"[{kategoria.upper()}]")

    # weryfikacja czy się znajdują nieodpowiednie treści
    czy_jest = None
    for i in range(len(zmiana_danych)):
        if "Nieodpowiednie treści" in zmiana_danych[i] and (zmiana_danych[i][1].lower() != "brak" or zmiana_danych[i][1].lower() != "nie"):
            # sprawdzanie, czy informacje są prawdziwe
            if double_check(user_message=user_message, data_to_check=zmiana_danych[i]) is True:
                zmiana_danych[i] = ("Nieodpowiednie treści", "Tak")
            else:
                zmiana_danych[i] = ("Nieodpowiednie treści", "Nie")
                czy_jest = True
                break
        elif "Nieodpowiednie treści" in zmiana_danych[i]:
            # sprawdzanie, czy rzeczywiście nie ma
            if double_check(user_message=user_message) is True:
                zmiana_danych[i] = ("Nieodpowiednie treści", "Tak")
            else:
                zmiana_danych[i] = ("Nieodpowiednie treści", "Nie")
                czy_jest = True
                break
        if "Nieodpowiednie treści" in zmiana_danych[i] and zmiana_danych[i][1] == "Tak":
            print("W tekście znajdują się nieodpowiednie treści")
            czy_jest = True
            break

    if czy_jest != True:  # jeżeli nie ma tego fragmentu to i tak sprawdza
        if double_check(user_message=user_message) is True:
            zmiana_danych.append(("Nieodpowiednie treści", "Tak"))
            print("W tekście znajdują się nieodpowiednie treści")
        else:
            zmiana_danych.append(("Nieodpowiednie treści", "Nie"))


    if len(zmiana_danych) > 1:
        print("Tekst zawiera dane wrażliwe")
    elif len(zmiana_danych) == 1:
        for item in zmiana_danych:
            if "Nieodpowiednie treści" not in item:
                print("Tekst zawiera dane wrażliwe")

    return zmiana_danych, user_message, przetworzony_tekst


In [12]:
ochrona("John Smith, urodzony 15 lipca 1985 roku w Nowym Jorku, mieszka na ulicy Maple Avenue 123. Jego numer telefonu to 555-123-4567, a adres e-mail to john.smith@example.com. Wczoraj wieczorem spotkał się z kolegami i zbyt wiele wypił, co skończyło się awanturą. John zachował się bardzo agresywnie i obraźliwie wobec innych osób, co jest niedopuszczalne. W wyniku incydentu został zatrzymany przez policję")

W tekście znajdują się nieodpowiednie treści
Tekst zawiera dane wrażliwe


([('Imię', 'John'),
  ('Nazwisko', 'Smith'),
  ('Data urodzenia', '15 lipca 1985'),
  ('Miejsce urodzenia', 'Nowy Jork'),
  ('Adres zamieszkania', 'Maple Avenue 123'),
  ('Numer telefonu', '555-123-4567'),
  ('Adres e-mail', 'john.smith@example.com'),
  ('Nieodpowiednie treści', 'Tak')],
 'John Smith, urodzony 15 lipca 1985 roku w Nowym Jorku, mieszka na ulicy Maple Avenue 123. Jego numer telefonu to 555-123-4567, a adres e-mail to john.smith@example.com. Wczoraj wieczorem spotkał się z kolegami i zbyt wiele wypił, co skończyło się awanturą. John zachował się bardzo agresywnie i obraźliwie wobec innych osób, co jest niedopuszczalne. W wyniku incydentu został zatrzymany przez policję',
 '[IMIĘ] [NAZWISKO], urodzony [DATA URODZENIA] roku w Nowym Jorku, mieszka na ulicy [ADRES ZAMIESZKANIA]. Jego numer telefonu to [NUMER TELEFONU], a adres e-mail to [ADRES E-MAIL]. Wczoraj wieczorem spotkał się z kolegami i zbyt wiele wypił, co skończyło się awanturą. [IMIĘ] zachował się bardzo [NIEODPOWI

## Limity
### Wykrywanie nieodpowiednich treści
Model jest przewrażliwiony i uznaje za dużo rzeczy za nieodpowiednie treści dlatego jest funkcja "double_check()", która sprawdza, czy to, co wykrył jest zgodne z tekstem czy nie
Zazwyczaj będzie poprawnie zwracał informację o występowaniu tych treści

### Zdrobnienia i odmiany
Model nie zawsze wypisuje wszystkich odmian imion, przez co zamiana tekstu na anonimowy nie jest możliwa, ponieważ używam do tego funkcji .replace()
Czasami wypisuje wersję słowa bez odmiany a w tekście jest odmieniona

### Precyzja
Im dłuższy tekst, tym mniejsza precyzja programu